<a href="https://colab.research.google.com/github/skylarvotaw/Math311_Linear_Algebra_Projects/blob/main/Copy_of_Sports_Teams_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the preliminary code

In [3]:
import matplotlib.pyplot as plt
from sympy import Poly
import sympy as sym
import numpy as np
from numpy import c_

# We'll specify that x and t are variables:
x, t = sym.symbols('x, t')

ll = "loaded"

In [4]:
lll = " correctly!"

def rref(matrix):
    A = np.array(matrix, dtype=np.float64)

    r = 0 #row
    c = 0 #column

    (maxr, maxc) = A.shape

    prev_row_pivot = -1   # The previous row with a leading term:
    tmp_pivot_row = -1
    this_pivot_row = -1

    for c in range(maxc):  # go through each column
      tmp_pivot_row = -1  # reset the tmp pivot row
      this_pivot_row = -1  # reset this pivot row
      for r in range(maxr):  # Find the leading term in this column below prev_row_pivot
        if ((r > prev_row_pivot) and (A[r][c] != 0.0)):
          tmp_pivot_row = r  # find the new pivot row
      if (tmp_pivot_row > -1):  # we found a candidate for the leading term:
        #swap row this_pivot_row with (prev_row_pivot + 1)
        A[[tmp_pivot_row, prev_row_pivot+1]] = A[[prev_row_pivot + 1, tmp_pivot_row]]
        #set this_pivot_row to prev_row_pivot+1
        this_pivot_row = prev_row_pivot + 1
        #Now, normalize the row based on the pivot value:
        A[ this_pivot_row ] = A[ this_pivot_row ] / A[ this_pivot_row ][c]
        #Then, use this row to force 0's above and below the pivot entry:
        #subtract multiples of of the this_pivot_row row from the others
        for r_ in range(len(A)):
            if r_ != this_pivot_row:
                A[r_] = A[r_] - A[this_pivot_row] * A[r_][c] / A[this_pivot_row][c]
        prev_row_pivot = this_pivot_row
    return A


def is_singular(matrix):
  singular = True
  notsingular = False

  (mr, mc) = matrix.shape

  return_value = notsingular
  if (mc - np.linalg.linalg.matrix_rank(matrix) > 0) :
    return_value = singular
  return return_value




print(">> The page ", ll, lll)

>> The page  loaded  correctly!


# Ranking Teams based on games

From our packet, we saw there were five games:

Home  | Away
------|------
Winona State 16 | Sioux Falls 15
Winona State 34 | Augustana 20
Sioux Falls 51  | Augustana 13
Duluth 28       |  Augustana 17
Winona State 17 | Duluth 37

Let's encode this system into a matrix.

The unknown rankings are $r_w, r_s, r_a, r_d$.

The system looks like $M \vec{x} = \vec{b}$, or

$$
\left( \begin{array}{cccc} 1 & -1 & 0 & 0 \\ 1 & 0 & -1 & 0 \\ 0 & 1 & -1 & 0 \\ 0 & 0 & -1 & 1 \\ -1 & 0  & 0 & 1 \\ \end{array} \right) \, \left( \begin{array}{c} r_w \\ r_s \\ r_a \\ r_d \\ \end{array} \right) = \left( \begin{array}{c} 1 \\ 14 \\ 38 \\ 11 \\ 20 \\ \end{array} \right)$$

We'll solve this system using least squares
$$M^T I M \vec{x} = M^T I \vec{b} $$

$$ \vec{x} = \left( M^T I M  \right)^{-1} M^T I \vec{b} $$






In [ ]:
# create the matrix:
M = np.matrix([[1, -1, 0, 0],
               [1, 0, -1, 0],
               [0, 1, -1, 0],
               [0, 0, -1, 1],
               [-1, 0, 0, 1]])


# create the vector for the right hand side:
pts = np.matrix([[1], [14], [38], [11], [20]])

#Make sure the system looks appropriate:
print(np.c_[M,pts])

[[ 1 -1  0  0  1]
 [ 1  0 -1  0 14]
 [ 0  1 -1  0 38]
 [ 0  0 -1  1 11]
 [-1  0  0  1 20]]


In [ ]:
# Generate the transpose of M:
MT = M.transpose()

print("The Transpose of M is:")
print(MT)

# and find MT*M and MT*pts:
print("\n")
print("MT * M is: ")
MTM = MT * M
print(MTM)

print("\n")
print("MT * pts is:")
MTpts = MT * pts
print(MTpts)


The Transpose of M is:
[[ 1  1  0  0 -1]
 [-1  0  1  0  0]
 [ 0 -1 -1 -1  0]
 [ 0  0  0  1  1]]


MT * M is: 
[[ 3 -1 -1 -1]
 [-1  2 -1  0]
 [-1 -1  3 -1]
 [-1  0 -1  2]]


MT * pts is:
[[ -5]
 [ 37]
 [-63]
 [ 31]]


If $M^T M$ has an inverse, then we can use it.  But, if $M^T M$ is singular, then it doesn't have an inverse.  In this situation, we need to replace every entry in one row (say the bottom row) of $M^T M$ and $M^T \vec{pts}$ with 1's.  (This knocks the system out of *not* having an inverse, so it will now have an inverse.)

In [ ]:
#  Check if MTM is singular
is_singular(MT*M)

True

In [ ]:
# If MTM is singular, then adjust one row of MTM and MT pts to be just 1's:

#MTM[3] is the entire 4th row of MTM,
#so MTM[3] = 1 will put 1's in MTM's 4th row.

MTM[3] = 1
MTpts[3] = 1

print("MTM is now: ")
print(MTM)

print("and MTpts is now:")
print(MTpts)

print("Is our modified MTM singular?")
is_singular(MTM)

MTM is now: 
[[ 3 -1 -1 -1]
 [-1  2 -1  0]
 [-1 -1  3 -1]
 [ 1  1  1  1]]
and MTpts is now:
[[ -5]
 [ 37]
 [-63]
 [  1]]
Is our modified MTM singular?


False

In [ ]:
#Find our solution using the inverse of MTM:

#First, find the inverse of MTM:
MTMI = np.linalg.inv(MTM)

print("The inverse of MTM is:")
print(MTMI)

#Now, use the inverse of MTM to solve for the rankings:

print("\n")
print("The solution are rankings:")
MTMI * MTpts

The inverse of MTM is:
[[ 2.50000000e-01  0.00000000e+00  1.85037171e-17  2.50000000e-01]
 [ 1.25000000e-01  5.00000000e-01  1.25000000e-01  2.50000000e-01]
 [ 0.00000000e+00  0.00000000e+00  2.50000000e-01  2.50000000e-01]
 [-3.75000000e-01 -5.00000000e-01 -3.75000000e-01  2.50000000e-01]]


The solution are rankings:


matrix([[ -1.  ],
        [ 10.25],
        [-15.5 ],
        [  7.25]])

#  Game Data  - Fall Sports

Find the game data for the four teams in the NFC North and NFC South.

The teams are:

* **NFC North:**  Green Bay Packers, Chicago Bears, Detroit Lions, Minnesota Vikings
* **NFC South:**  Atlanta Falcons, Carolina Panthers, New Orleans Saints, Tampa Bay Buccaneers


It may be helpful to do a google search to find the data.

Be sure to create your matrix on the handout, then enter the data for the system below.

# Game Data - Spring Sports

Find the game data for the seven teams in the Western Division for Men's Big Ten Basketball.

**The teams:**  Wisconsin, Minnesota, Iowa, Illinois, Nebraska, Purdue, and Northewstern.

I've provided **most** of the data, but you have to do a Google search to find the scores for the last two teams.

Be sure to create your matrix on the handout, then enter the data for the system below.


In [1]:
[1,-1,0,0,0,0,0,0],
[-1,0,0,0,0,1,0,0],
[-1,0,0,0,0,0,1,0],
[1,0,0,0,0,0,0,-1],
[-1,0,0,1,0,0,0,0],
[-1,0,1,0,0,0,0,0],
[-1,0,0,0,1,0,0,0],
[0,0,0,0,1,-1,0,0],
[0,0,1,0,-1,0,0,0],
[0,0,0,1,-1,0,0,0],
[0,-1,0,1,0,0,0,0],
[0,-1,0,0,0,0,1,0],
[0,-1,1,0,0,0,0,0],
[0,-1,0,0,0,1,0,0],
[0,0,1,0,0,0,0,-1],
[0,0,1,0,0,-1,0,0],
[0,0,-1,0,0,0,1,0],
[0,0,0,-1,0,0,1,0],
[0,0,0,1,0,-1,0,0],
[0,0,0,0,1,0,-1,0],
[0,0,0,0,0,0,1,-1]

[0, 0, 0, 0, 0, 0, 1, -1]

In [6]:
# create the matrix:
M = np.matrix([[1,-1,0,0,0,0,0,0],
               [-1,0,0,0,0,1,0,0],
               [-1,0,0,0,0,0,1,0],
               [1,0,0,0,0,0,0,-1],
               [-1,0,0,1,0,0,0,0],
               [-1,0,1,0,0,0,0,0],
               [-1,0,0,0,1,0,0,0],
               [0,0,0,0,1,-1,0,0],
               [0,0,1,0,-1,0,0,0],
               [0,0,0,1,-1,0,0,0],
               [0,-1,0,1,0,0,0,0],
               [0,-1,0,0,0,0,1,0],
               [0,-1,1,0,0,0,0,0],
               [0,-1,0,0,0,1,0,0],
               [0,0,1,0,0,0,0,-1],
               [0,0,1,0,0,-1,0,0],
               [0,0,-1,0,0,0,1,0],
               [0,0,0,-1,0,0,1,0],
               [0,0,0,1,0,-1,0,0],
               [0,0,0,0,1,0,-1,0],
               [0,0,0,0,0,0,1,-1]])


# create the vector for the right hand side:
pts = np.matrix([[3], [3], [2], [3], [2], [3], [2], [1], [3], [3], [3], [3], [2], [3], [3], [1], [3], [3], [2], [3], [3] ])

#Make sure the system looks appropriate:
print(np.c_[M,pts])

[[ 1 -1  0  0  0  0  0  0  3]
 [-1  0  0  0  0  1  0  0  3]
 [-1  0  0  0  0  0  1  0  2]
 [ 1  0  0  0  0  0  0 -1  3]
 [-1  0  0  1  0  0  0  0  2]
 [-1  0  1  0  0  0  0  0  3]
 [-1  0  0  0  1  0  0  0  2]
 [ 0  0  0  0  1 -1  0  0  1]
 [ 0  0  1  0 -1  0  0  0  3]
 [ 0  0  0  1 -1  0  0  0  3]
 [ 0 -1  0  1  0  0  0  0  3]
 [ 0 -1  0  0  0  0  1  0  3]
 [ 0 -1  1  0  0  0  0  0  2]
 [ 0 -1  0  0  0  1  0  0  3]
 [ 0  0  1  0  0  0  0 -1  3]
 [ 0  0  1  0  0 -1  0  0  1]
 [ 0  0 -1  0  0  0  1  0  3]
 [ 0  0  0 -1  0  0  1  0  3]
 [ 0  0  0  1  0 -1  0  0  2]
 [ 0  0  0  0  1  0 -1  0  3]
 [ 0  0  0  0  0  0  1 -1  3]]


In [7]:
# Generate the transpose of M:
MT = M.transpose()

print("The Transpose of M is:")
print(MT)

# and find MT*M and MT*pts:
print("\n")
print("MT * M is: ")
MTM = MT * M
print(MTM)

print("\n")
print("MT * pts is:")
MTpts = MT * pts
print(MTpts)


The Transpose of M is:
[[ 1 -1 -1  1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  1  0  0  0  1  0  1  1 -1  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  1  1  0  0  0  0  0  0 -1  1  0  0]
 [ 0  0  0  0  0  0  1  1 -1 -1  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  1  0  0  0  0  0 -1  0  0  0  0  0  1  0 -1  0  0 -1  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  1  1  0 -1  1]
 [ 0  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0 -1]]


MT * M is: 
[[ 7 -1 -1 -1 -1 -1 -1 -1]
 [-1  5 -1 -1  0 -1 -1  0]
 [-1 -1  6  0 -1 -1 -1 -1]
 [-1 -1  0  5 -1 -1 -1  0]
 [-1  0 -1 -1  5 -1 -1  0]
 [-1 -1 -1 -1 -1  5  0  0]
 [-1 -1 -1 -1 -1  0  6 -1]
 [-1  0 -1  0  0  0 -1  3]]


MT * pts is:
[[ -6]
 [-14]
 [  9]
 [  7]
 [  0]
 [  2]
 [ 11]
 [ -9]]


In [8]:
#  Check if MTM is singular
is_singular(MT*M)

/tmp/ipython-input-4116023008.py:2: DeprecationWarning: The numpy.linalg.linalg has been made private and renamed to numpy.linalg._linalg. All public functions exported by it are available from numpy.linalg. Please use numpy.linalg.matrix_rank instead.
  is_singular(MT*M)


True

In [9]:
# Careful - you might not need to do this!
# Be sure to check whether MTM is singular on the previous line.

# If MTM is singular, then adjust one row of MTM and MT pts to be just 1's:

#MTM[3] is the entire 4th row of MTM,
#so MTM[3] = 1 will put 1's in MTM's 4th row.

MTM[3] = 1
MTpts[3] = 1

print("MTM is now: ")
print(MTM)

print("and MTpts is now:")
print(MTpts)

print("Is our modified MTM singular?")
is_singular(MTM)

MTM is now: 
[[ 7 -1 -1 -1 -1 -1 -1 -1]
 [-1  5 -1 -1  0 -1 -1  0]
 [-1 -1  6  0 -1 -1 -1 -1]
 [ 1  1  1  1  1  1  1  1]
 [-1  0 -1 -1  5 -1 -1  0]
 [-1 -1 -1 -1 -1  5  0  0]
 [-1 -1 -1 -1 -1  0  6 -1]
 [-1  0 -1  0  0  0 -1  3]]
and MTpts is now:
[[ -6]
 [-14]
 [  9]
 [  1]
 [  0]
 [  2]
 [ 11]
 [ -9]]
Is our modified MTM singular?


/tmp/ipython-input-2134225523.py:19: DeprecationWarning: The numpy.linalg.linalg has been made private and renamed to numpy.linalg._linalg. All public functions exported by it are available from numpy.linalg. Please use numpy.linalg.matrix_rank instead.
  is_singular(MTM)


False

In [10]:
#Find our solution using the inverse of MTM:

#First, find the inverse of MTM:
MTMI = np.linalg.inv(MTM)

print("The inverse of MTM is:")
print(MTMI)

#Now, use the inverse of MTM to solve for the rankings:

print("\n")
print("The solution are rankings:")
MTMI * MTpts

The inverse of MTM is:
[[ 1.25000000e-01  3.96508223e-18  4.21289987e-18  1.25000000e-01
  -5.94762335e-18 -4.18374485e-18 -2.23035875e-18  1.98254112e-18]
 [-7.23140496e-03  1.70247934e-01 -8.26446281e-03  1.25000000e-01
  -2.97520661e-02 -5.00008531e-18 -8.26446281e-03 -4.95867769e-02]
 [ 2.56248740e-02  2.45918162e-02  1.72142713e-01  1.25000000e-01
   2.45918162e-02  2.43902439e-02  2.58012497e-02  3.28562790e-02]
 [-1.79374118e-01 -1.72142713e-01 -2.04998992e-01  1.25000000e-01
  -1.72142713e-01 -1.70731707e-01 -1.80608748e-01 -2.29993953e-01]
 [-7.23140496e-03 -2.97520661e-02 -8.26446281e-03  1.25000000e-01
   1.70247934e-01  6.08657306e-18 -8.26446281e-03 -4.95867769e-02]
 [-8.64241080e-03 -1.41100585e-03 -9.87704092e-03  1.25000000e-01
  -1.41100585e-03  1.70731707e-01 -3.42672848e-02 -5.92622455e-02]
 [ 1.23463011e-03  2.01572264e-04  1.41100585e-03  1.25000000e-01
   2.01572264e-04 -2.43902439e-02  1.47752469e-01  8.46603507e-03]
 [ 5.06198347e-02  8.26446281e-03  5.78512397e

matrix([[-0.625     ],
        [-1.93409091],
        [ 1.21313747],
        [ 1.50803769],
        [ 0.86590909],
        [ 0.60559867],
        [ 1.62777162],
        [-2.26136364]])

The solution are rankings:
matrix([[-0.625     ], Winona
        [-1.93409091], Aug
        [ 1.21313747], Minn State
        [ 1.50803769], Wayne
        [ 0.86590909], Sioux
        [ 0.60559867], Southwest
        [ 1.62777162], Concordia
        [-2.26136364]]) UMary

Winona will beat UMary by 1.6 Sets. This means Winona will win 3-1.
